In [1]:
from functools import reduce
import math
import os
import string
import random

from datasets import load_dataset
from tqdm import tqdm
import torch

from config import GenericArgs
from utils.misc import compute_ber, riskset, stop
from utils.contextls_utils import synchronicity_test, substitutability_test, tokenizer, riskset, stop
from utils.logging import getLogger
from utils.dataset_utils import preprocess_txt, preprocess2sentence, get_result_txt, get_dataset
from utils.metric import Metric

import json

random.seed(1230)

Error importing yake


BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at /root/autodl-tmp/bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.de

In [2]:
def get_list(string):
    output = []
    string = string.replace(",", "")
    str_list = list(string.replace(" ", ""))
    for i in str_list:
        output.append(int(i))
    return output

def main(cover_text, f, extracting=False):
    """
    cover_text: 待处理的单个句子
    """
    encoded_text = tokenizer(cover_text, add_special_tokens=False, truncation=True,
                             max_length=tokenizer.model_max_length // 2 - 2)    # 编码
    word_ids = encoded_text._encodings[0].word_ids   # 获取句子中每个单词对应的id
    watermarking_wordset = [None] * len(encoded_text['input_ids'])   # 水印词集
    substituted_idset = []   # 被替换的词的id集合
    substituted_indices = []   # 被替换的索引
    watermarked_text = []   # 水印文本
    message = []   # 嵌入的信息

    latest_embed_index = -1
    index = 1
    while index < len(encoded_text['input_ids']) - f:   # 从第二个词开始处理，直到划定的f为止
        text = tokenizer.decode(encoded_text['input_ids'][index])  # 还原当前处理的词
        watermarked_text.append(text)   # 

        if text in riskset:   # 若该词在风险集中，就不处理该词
            watermarking_wordset[index] = 'riskset'
            index = index + 1
            continue

        valid_indx = [t == index for t in word_ids]
        if sum(valid_indx) >= 2:  # skip this word; subword
            watermarking_wordset[index] = 'subword'
            index = index + 1
            continue

        local_context = encoded_text['input_ids'][:index + 2]  # 获取当前词的局部上下文
        is_synch, words = synchronicity_test(index, local_context)   # 做同步性测试

        if not is_synch:  # 不具备同步性，跳过该词
            watermarking_wordset[index] = 'syn'
            index = index + 1
            continue

        if index - latest_embed_index != f + 1:   # 当前词与上一个词之间的距离不超过f，做可替换性测试
            if not substitutability_test(local_context[:-1], index,
                                         words):  # skip this word if substitutability test fails
                watermarking_wordset[index] = 'sub'    # 没通过可替换性测试，跳过该词
                index = index + 1
                continue

        watermarking_wordset[index] = tokenizer.decode(words)
        words_decoded = tokenizer.decode(words).split(" ")
        # sort token_id alphabetically
        words = [w for w, wd in sorted(zip(words, words_decoded), key=lambda pair: pair[1])]
        words_decoded.sort()

        # extraction
        if extracting:
            extracted_msg = words_decoded.index(text)
            message.append(extracted_msg)

        # embedding
        else:
            random_msg = random.choice([0,1])
            word_chosen_by_msg = words[random_msg]
            encoded_text['input_ids'][index] = word_chosen_by_msg
            message.append(random_msg)


        substituted_idset.append(words)
        substituted_indices.append(index)
        latest_embed_index = index
        index = index + f + 1

    return substituted_idset, substituted_indices, watermarking_wordset, encoded_text['input_ids'], message

def extract(dataset, name, f=1):
    print(f'======{name}提取水印======')
    z_score_result = []

    corrupted_watermarked = []


    num_corrupted_sen = 0
    sample_level_bit = {'gt':[], 'extracted':[]}
    bit_error_agg = {}
    midx_match_cnt = 0
    mword_match_cnt = 0
    infill_match_cnt = 0
    prev_c_idx = 0
        
    for index in tqdm(range(len(dataset))):
        text_index = index
        sentence_length = len(dataset[index])
        bit_error_agg = {"sentence_err_cnt": 0, "sentence_cnt": 0}
        for idx in range(len(dataset[index])):  # 处理每个句子
            data = dataset[index][idx]
            c_idx = data['c_idx']
            sen_idx = data['sen_idx']
            sub_idset = data['s_idset_str']
            sub_idx = data['s_indices_str']
            clean_wm_sen = data['clean_wm_text']
            key = data['keys_str']
            msg = data['message_str']

            if prev_c_idx != c_idx:
                error_cnt, cnt = compute_ber(sample_level_bit['extracted'], sample_level_bit['gt'])
                bit_error_agg['sample_err_cnt'] = bit_error_agg.get('sample_err_cnt', 0) + error_cnt
                bit_error_agg['sample_cnt'] = bit_error_agg.get('sample_cnt', 0) + cnt
                sample_level_bit = {'gt': [], 'extracted': []}
                prev_c_idx = c_idx

            if len(corrupted_watermarked) > 0 and len(corrupted_watermarked) <= idx:
                break

            clean_encoded = tokenizer(clean_wm_sen.strip(), add_special_tokens=False, truncation=True,
                                      max_length=tokenizer.model_max_length// 2 - 2)['input_ids']
            
            wm_texts = [clean_wm_sen.strip()]
            for wm_text in wm_texts:
                wm_text = wm_text.strip()

                num_corrupted_sen += 1
                extracted_idset, extracted_indices, watermarking_wordset, encoded_text, extracted_msg = \
                    main(wm_text, f, extracting=True)
                extracted_key = [tokenizer.decode(s_id) for s_id in extracted_idset]

                midx_match_flag = set(extracted_indices) == set(sub_idx)
                if midx_match_flag:
                    midx_match_cnt += 1

                mword_match_flag = set([encoded_text[idx] for idx in extracted_indices]) == \
                                   set([clean_encoded[idx] for idx in sub_idx])
                if mword_match_flag:
                    mword_match_cnt += 1

                infill_match_list = []
                if len(sub_idset) == len(extracted_idset):
                    for a, b in zip(sub_idset, extracted_idset):
                        infill_match_flag = a==b
                        infill_match_list.append(infill_match_flag)
                else:
                    infill_match_list.append(False)
                if all(infill_match_list):
                    infill_match_cnt += 1

                sample_level_bit['extracted'].extend(extracted_msg)
                sample_level_bit['gt'].extend(msg)
                error_cnt, cnt = compute_ber(msg, extracted_msg)
                bit_error_agg['sentence_err_cnt'] = bit_error_agg.get('sentence_err_cnt', 0) + error_cnt
                bit_error_agg['sentence_cnt'] = bit_error_agg.get('sentence_cnt', 0) + cnt

                match_flag = error_cnt == 0

        # 做z-test
        ones = bit_error_agg['sentence_cnt'] - bit_error_agg['sentence_err_cnt']
        n = bit_error_agg['sentence_cnt']
        p = 0.5  # Null hypothesis: perfect extraction (BER = 0)
        if n == 0:
            z_score = 0 
        else:
            z_score = (ones - p * n) / (n * p * (1 - p)) ** 0.5
        z_score_result.append({
            'text-index': text_index,
            'sentence-length': sentence_length,
            'z-score': z_score
        })
    print(f'=========={name}完成==========')
    return z_score_result

def embed(file_name, original_file, watermark_file):
    f = 1
    start_sample_idx = 0

    # corpus: 数据集
    dataset = json.load(open(file_name))
    # print(len(dataset2))
    # print(f'==========\n{dataset2[0]}\n==========')
    # cover_texts: 待处理的数据集
    cover_texts = preprocess2sentence(preprocess_txt(dataset), corpus_name="custom", start_sample_idx=0, cutoff_q=(0.0, 1.0), use_cache=False)

    # num_sentence = 0
    # for c_idx, sentences in enumerate(cover_texts):
    #     num_sentence += len(sentences)
    #     if num_sentence >= args.num_sample:
    #         break
    # cover_texts = cover_texts[:c_idx + 1]

    bit_count = 0
    word_count = 0

    print(f'=========={len(cover_texts)}==========')
    # print(cover_texts)

    # 嵌入水印
    original_output = []
    watermark_output = []
    for c_idx, sentences in enumerate(tqdm(cover_texts)):  # setences: 每一段文本
        original_result = []
        watermark_result = []
        for sen_idx, sen in enumerate(sentences):    # 每一个句子
            # print(f'======sen:{sen_idx}======')
            sen_text = sen.text    
            substituted_idset, substituted_indices, watermarking_wordset, encoded_text, message = main(sen_text, f)
            punct_removed = sen_text.translate(str.maketrans(dict.fromkeys(string.punctuation)))
            word_count += len([i for i in punct_removed.split(" ") if i not in stop])
            num_cases = 1
            for sid in substituted_idset:
                num_cases *= len(sid)
            bit_count += math.log2(num_cases)
            s_idset_str = ""
            for s_id in substituted_idset:
                s_idset_str += " ".join(str(x) for x in s_id) + ","
            s_indices_str = " ".join(str(x) for x in substituted_indices)
            message_str = [str(m) for m in message]
            message_str = " ".join(message_str) if len(message_str) else ""
            watermarked_text = tokenizer.decode(encoded_text)
            keys = [tokenizer.decode(s_id) for s_id in substituted_idset]
            keys_str = ", ".join(keys)
            # wr.write(f"{c_idx}\t{sen_idx}\t{s_idset_str}\t{s_indices_str}\t"
            #              f"{watermarked_text}\t{keys_str}\t{message_str}\n")
            message_str = get_list(message_str)
            s_idset_str = get_list(s_idset_str)
            s_indices_str = get_list(s_indices_str)
            watermark_result.append({
                'c_idx': c_idx,
                'sen_idx': sen_idx,
                's_idset_str': s_idset_str,
                's_indices_str': s_indices_str,
                'clean_wm_text': watermarked_text,
                'keys_str': keys_str,
                'message_str': message_str
            })
            original_result.append({
                'c_idx': c_idx,
                'sen_idx': sen_idx,
                's_idset_str': s_idset_str,
                's_indices_str': s_indices_str,
                'clean_wm_text': sen.text,
                'keys_str': keys_str,
                'message_str': message_str
            })
        original_output.append(original_result)
        watermark_output.append(watermark_result)

    # print(original_output[0])

    os.makedirs(os.path.dirname(original_file), exist_ok=True)
    with open(original_file, 'w', encoding='utf-8') as f:
        json.dump(original_output, f, indent=4, ensure_ascii=False)

    os.makedirs(os.path.dirname(watermark_file), exist_ok=True)
    with open(watermark_file, 'w', encoding='utf-8') as f:
        json.dump(watermark_output, f, indent=4, ensure_ascii=False)

    return original_output, watermark_output

# bloomz

In [ ]:
file_name_bloomz = f'data/M4/arxiv/abstract/dataset-M4-random-bloomz-800.json'
original_file_bloomz = f'result/ContextLS/M4/original/bloomz-random-800.json'
watermark_file_bloomz = f'result/ContextLS/M4/watermark/bloomz-random-800.json'
z_score_file_bloomz = f'result/ContextLS/M4/z_score/bloomz-random-800.json'

original_output_bloomz, watermark_output_bloomz = embed(file_name_bloomz, original_file_bloomz, watermark_file_bloomz)

ori_fast_z_score_bloomz = extract(original_output_bloomz, '原始文本')
water_fast_z_score_bloomz = extract(watermark_output_bloomz, '水印文本')

z_score_output_bloomz = []

for index in range(len(ori_fast_z_score_bloomz)):
    z_score_output_bloomz.append({
        'text-index': ori_fast_z_score_bloomz[index]['text-index'],
        'sentence-length': ori_fast_z_score_bloomz[index]['sentence-length'],
        'ori-fast-z-score': ori_fast_z_score_bloomz[index]['z-score'],
        'water-fast-z-score': water_fast_z_score_bloomz[index]['z-score']

    })

os.makedirs(os.path.dirname(z_score_file_bloomz), exist_ok=True)
with open(z_score_file_bloomz, 'w', encoding='utf-8') as f:
    json.dump(z_score_output_bloomz, f, indent=4, ensure_ascii=False)

==========800==========


 65%|██████▍   | 517/800 [8:07:22<5:58:00, 75.90s/it]  

# ChatGPT

In [ ]:
file_name_chatGPT = f'data/M4/arxiv/abstract/dataset-M4-random-chatGPT-800.json'
original_file_chatGPT = f'result/ContextLS/M4/original/chatGPT-random-800.json'
watermark_file_chatGPT = f'result/ContextLS/M4/watermark/chatGPT-random-800.json'
z_score_file_chatGPT = f'result/ContextLS/M4/z_score/chatGPT-random-800.json'

original_output_chatGPT, watermark_output_chatGPT = embed(file_name_chatGPT, original_file_chatGPT, watermark_file_chatGPT)

ori_fast_z_score_chatGPT = extract(original_output_chatGPT, '原始文本')
water_fast_z_score_chatGPT = extract(watermark_output_chatGPT, '水印文本')

z_score_output_chatGPT = []

for index in range(len(ori_fast_z_score_chatGPT)):
    z_score_output_chatGPT.append({
        'text-index': ori_fast_z_score_chatGPT[index]['text-index'],
        'sentence-length': ori_fast_z_score_chatGPT[index]['sentence-length'],
        'ori-fast-z-score': ori_fast_z_score_chatGPT[index]['z-score'],
        'water-fast-z-score': water_fast_z_score_chatGPT[index]['z-score']

    })

os.makedirs(os.path.dirname(z_score_file_chatGPT), exist_ok=True)
with open(z_score_file_chatGPT, 'w', encoding='utf-8') as f:
    json.dump(z_score_output_chatGPT, f, indent=4, ensure_ascii=False)

# cohere

In [ ]:
dataset_name = ['bloomz', 'chatGPT', 'cohere', 'davinci', 'dolly', 'flant5']

file_name_cohere = f'data/M4/arxiv/abstract/dataset-M4-random-cohere-800.json'
original_file_cohere = f'result/ContextLS/M4/original/cohere-random-800.json'
watermark_file_cohere = f'result/ContextLS/M4/watermark/cohere-random-800.json'
z_score_file_cohere = f'result/ContextLS/M4/z_score/cohere-random-800.json'

original_output_cohere, watermark_output_cohere = embed(file_name_cohere, original_file_cohere, watermark_file_cohere)

ori_fast_z_score_cohere = extract(original_output_cohere, '原始文本')
water_fast_z_score_cohere = extract(watermark_output_cohere, '水印文本')

z_score_output_cohere = []

for index in range(len(ori_fast_z_score_cohere)):
    z_score_output_cohere.append({
        'text-index': ori_fast_z_score_cohere[index]['text-index'],
        'sentence-length': ori_fast_z_score_cohere[index]['sentence-length'],
        'ori-fast-z-score': ori_fast_z_score_cohere[index]['z-score'],
        'water-fast-z-score': water_fast_z_score_cohere[index]['z-score']

    })

os.makedirs(os.path.dirname(z_score_file_cohere), exist_ok=True)
with open(z_score_file_cohere, 'w', encoding='utf-8') as f:
    json.dump(z_score_output_cohere, f, indent=4, ensure_ascii=False)